In [2]:
from scipy import stats
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas_profiling as pp
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import precision_score, roc_auc_score, make_scorer, f1_score, recall_score, roc_auc_score
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
import os
os.environ['PYTHONHASHSEED']=str(271828)
random.seed(271828)
np.random.seed(271828)
#import tensorflow as tf
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from tensorflow.python.framework import ops
import tensorflow as tf
tf.random.set_seed(271828)
ops.reset_default_graph()
from fancyimpute import IterativeImputer

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
#from tensorflow.keras.layers import Conv2D, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping

import keras.backend as K
#session_conf=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1) 
#sesCriticalSection=tf.Session(graph=tf.get_default_graph(), config=session_conf)
#K.set_session(sess)

Using TensorFlow backend.


In [3]:
def PCL_calculator(df):

    symptomatic_cutoff = 2

    intrusion = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'q6.4_UPSET', 'q6.5_PHYS']
    avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES', 'q6.9_DISINT', 'q6.10_DTACH',
                 'q6.11_NUMB', 'q6.12_FUTRE']
    tred = ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH']
    only_avoidance = ['q6.6_AVTHT', 'q6.7_AVSIT', 'q6.8_AMNES']

    hypertension = ['q6.13_SLEEP', 'q6.14_ANGER', 'q6.15_CONC', 'q6.16_HYPER', 'q6.17_STRTL']

    depression = ['q6.9_DISINT', 'q6.10_DTACH', 'q6.11_NUMB', 'q6.12_FUTRE']

    df[intrusion + avoidance + hypertension].fillna(df[intrusion + avoidance + hypertension].mean(axis=1))
    intrusion_cuoff = 1
    avoidance_cuoff = 3
    hypertension_cuoff = 2
    only_avoidance_cutoff = 1
    depression_cutoff = 2
    tred_cutoff = 1

    df['sum'] = (df[intrusion + avoidance + hypertension]).sum(axis=1)

    df['intrusion'] = (df[intrusion] > symptomatic_cutoff).sum(axis=1)
    df['intrusion_cutoff'] = (df['intrusion'] >= intrusion_cuoff).astype(int)

    df['avoidance'] = (df[avoidance] > symptomatic_cutoff).sum(axis=1)
    df['avoidance_cutoff'] = (df['avoidance'] >= avoidance_cuoff).astype(int)

    df['depression'] = (df[depression] > symptomatic_cutoff).sum(axis=1)
    df['depression_cutoff'] = df['depression'] >= depression_cutoff

    df['hypertention'] = (df[hypertension] > symptomatic_cutoff).sum(axis=1)
    df['hypertention_cutoff'] = (df['hypertention'] >= hypertension_cuoff).astype(int)

    df['tred'] = (df[tred] > symptomatic_cutoff).sum(axis=1)
    df['tred_cutoff'] = df['tred'] >= tred_cutoff

    df['only_avoidance'] = (df[only_avoidance] > symptomatic_cutoff).sum(axis=1)
    df['only_avoidance_cutoff'] = df['only_avoidance'] >= only_avoidance_cutoff

    df['regression_cutoff_33'] = df['sum'] >= 33
    df['regression_cutoff_50'] = df['sum'] >= 49
    df['diagnosis'] = ((df['hypertention_cutoff']) & (df['avoidance_cutoff']) & (df['intrusion_cutoff']) & (df['sum'] >= 49))

    return df


In [4]:
path = "C:\‏‏PycharmProjects\PTSD\Data\PTSD.xlsx"
df = pd.read_excel(path)
#combine with specifics of answers
df_pcl2 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL2.xlsx")
df_pcl1 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL1.xlsx")
df = df.merge(df_pcl1, on="ID", how='outer')
df = df.merge(df_pcl2, suffixes=('_pcl1', '_pcl2'), on="ID", how='outer')

df_pcl3 = pd.read_excel("C:\‏‏PycharmProjects\PTSD\Data\questionnaire6PCL3.xlsx")
df_pcl3 = PCL_calculator(df_pcl3)
df = df.merge(df_pcl3.drop(['PCL3_Strict', 'pcl3', 'PCL3_Broad'], axis=1), on="ID", how='outer')
# rmoving missing Y's
df = df[~df['tred_cutoff'].isna()]

In [5]:
# removed age
features = ["highschool_diploma", "dyslexia", "ADHD", "Ashken_scale", "Sephar_scale", "Unknown_scale",
            "T1Acc1t", "T1Acc1n", "T1bias", "phq1", "lot1", "trait1", "state1", "PCL1",
            "PCL_Broad1", "PCL_Strict1", "phq2", "lot2", "trait2", "state2", "PCL2",
            "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1", "planning1",
            "positive_reframing1", "acceptance1", "humor1", "religion1", "emotional_support1",
            "instrumental_support1", "self_distraction1", "denial1", "venting1", "substance_use1",
            "behavioral_disengagement1", "self_blame1", "active_coping2", "planning2",
                "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                "instrumental_support2", "self_blame2", "self_distraction2", "denial2",
            "venting2", "substance_use2", "behavioral_disengagement2",
                "trauma_history8_1", "HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", 'q6.1_INTRU_pcl1', 
                'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                  'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                  'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                  'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                  'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                  'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                  'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                  'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2']

In [6]:
t2_features = [
    "lot2", "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "phq2",

    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2", 
    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
    
    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2', 'q6.5_PHYS_pcl2',
    'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2'
]
t1_features =[ "phq1", "lot1", "trait1","state1", "PCL1", "PCL_Broad1", "PCL_Strict1",
              
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1']

In [7]:
discrete_features = ["highschool_diploma", "dyslexia", "ADHD", "phq1", "lot1",
                    "trait1", "state1", "PCL1", "PCL_Broad1", "PCL_Strict1", "phq2", "lot2",
                    "trait2", "state2", "PCL2", "PCL_Broad2", "PCL_Strict2", "cd_risc1", "active_coping1",
                    "planning1", "positive_reframing1", "acceptance1", "humor1", "religion1",
                    "emotional_support1", "instrumental_support1", "self_distraction1", "denial1",
                    "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1", "active_coping2",
                    "planning2", "positive_reframing2", "acceptance2", "humor2", "religion2", "emotional_support2",
                    "instrumental_support2", "self_distraction2", "denial2", "venting2", "substance_use2",
                    "behavioral_disengagement2", "self_blame2", "trauma_history8_1",
                    'q6.1_INTRU_pcl1', 'q6.2_DREAM_pcl1', 'q6.3_FLASH_pcl1', 'q6.4_UPSET_pcl1',
                    'q6.5_PHYS_pcl1', 'q6.6_AVTHT_pcl1', 'q6.7_AVSIT_pcl1', 'q6.8_AMNES_pcl1', 'q6.9_DISINT_pcl1',
                    'q6.10_DTACH_pcl1', 'q6.11_NUMB_pcl1', 'q6.12_FUTRE_pcl1', 'q6.13_SLEEP_pcl1',
                    'q6.14_ANGER_pcl1', 'q6.15_CONC_pcl1', 'q6.16_HYPER_pcl1', 'q6.17_STRTL_pcl1',
                    'q6.1_INTRU_pcl2', 'q6.2_DREAM_pcl2', 'q6.3_FLASH_pcl2', 'q6.4_UPSET_pcl2',
                    'q6.5_PHYS_pcl2', 'q6.6_AVTHT_pcl2', 'q6.7_AVSIT_pcl2', 'q6.8_AMNES_pcl2', 'q6.9_DISINT_pcl2',
                    'q6.10_DTACH_pcl2', 'q6.11_NUMB_pcl2', 'q6.12_FUTRE_pcl2', 'q6.13_SLEEP_pcl2',
                    'q6.14_ANGER_pcl2', 'q6.15_CONC_pcl2', 'q6.16_HYPER_pcl2', 'q6.17_STRTL_pcl2']

In [8]:
genome_features = ["HML_5HTT", "HL_MAOA", "HML_NPY", "COMT_Hap1_recode",
                "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5"]
continuous_features = ["T1Acc1t", "T1Acc1n", "T1bias"]
demographic_features = ["highschool_diploma", "dyslexia", "ADHD", "Ashken_scale", "Sephar_scale", "Unknown_scale"]

In [9]:
df1 = df[features + ['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'tred_cutoff']]
#df1 = df1.dropna(thresh=47)
df_t1 = df1[t1_features]
df_t2 = df1[t2_features]
df1 = df1[(df_t1.isna().astype(int).sum(axis=1)<38) | (df_t2.isna().astype(int).sum(axis=1)<38)]
df1["t1_missing"] = df_t1.isna().astype(int).sum(axis=1)==38
df1["t2_missing"] = df_t2.isna().astype(int).sum(axis=1)==38
features.extend(["t1_missing", "t2_missing"])


In [10]:
X, X_out, Y, y_out = train_test_split(df1[features], df1[['q6.1_INTRU', 'q6.2_DREAM', 'q6.3_FLASH', 'tred_cutoff']],
                                      test_size=0.25, random_state=271828, stratify=df1['tred_cutoff'])

In [11]:
impute = 1
if impute:
    mice = IterativeImputer()
    X = pd.DataFrame(mice.fit_transform(X), columns=X.columns)
    Y = Y.fillna(Y.mean())
    features.extend(["outliers_count_t1_35", "outliers_count_t2_35", "outliers_count_t1_25", "outliers_count_t2_25"])
    X["outliers_count_t1_35"] = (np.abs(stats.zscore(X[t1_features])) > 3.5).sum(axis=1)
    X["outliers_count_t1_25"] = (np.abs(stats.zscore(X[t1_features])) > 2.5).sum(axis=1)
    X["outliers_count_t2_35"] = (np.abs(stats.zscore(X[t2_features])) > 3.5).sum(axis=1)
    X["outliers_count_t2_25"] = (np.abs(stats.zscore(X[t2_features])) > 2.5).sum(axis=1)
    
    

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y['tred_cutoff'], random_state=271828)
cv = StratifiedKFold(3, random_state=271828)
scores_f = []
scores_p = []
scores_r = []
scores_auc = []

for train, test in cv.split(X_train, y_train['tred_cutoff'].astype(int)):

    X_train_cv = X_train.iloc[train]
    y_train_cv = y_train.iloc[train]

    t = [y_train_cv['q6.1_INTRU'], y_train_cv['q6.2_DREAM'],
         y_train_cv['q6.3_FLASH'], y_train_cv['tred_cutoff']]
    
    y_pred_1 = CatBoostClassifier().fit(X_train_cv, (t[0]>1).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_2 = CatBoostClassifier().fit(X_train_cv, (t[1]>1).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_3 = CatBoostClassifier().fit(X_train_cv, (t[2]>1).values.astype(int)).predict_proba(X_train.iloc[test])
    y_pred_target = CatBoostClassifier().fit(X_train_cv, t[3]).predict_proba(X_train.iloc[test])

    y_pred = (y_pred_1[:, 1] + y_pred_2[:, 1] +  y_pred_3[:, 1]) > 0.5 #+ 0.5 * y_pred_target[:, 1]) > 1

    scores_f.append(f1_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_p.append(precision_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_r.append(recall_score(y_train['tred_cutoff'].iloc[test], y_pred))
    scores_auc.append(roc_auc_score(y_train['tred_cutoff'].iloc[test], y_pred))

Learning rate set to 0.011481
0:	learn: 0.6874025	total: 238ms	remaining: 3m 57s
1:	learn: 0.6831244	total: 408ms	remaining: 3m 23s
2:	learn: 0.6786680	total: 579ms	remaining: 3m 12s
3:	learn: 0.6731663	total: 738ms	remaining: 3m 3s
4:	learn: 0.6683062	total: 887ms	remaining: 2m 56s
5:	learn: 0.6651769	total: 1.04s	remaining: 2m 52s
6:	learn: 0.6617833	total: 1.19s	remaining: 2m 49s
7:	learn: 0.6586185	total: 1.34s	remaining: 2m 46s
8:	learn: 0.6544241	total: 1.51s	remaining: 2m 46s
9:	learn: 0.6505250	total: 1.66s	remaining: 2m 44s
10:	learn: 0.6473237	total: 1.77s	remaining: 2m 39s
11:	learn: 0.6443046	total: 1.92s	remaining: 2m 38s
12:	learn: 0.6387226	total: 2.06s	remaining: 2m 36s
13:	learn: 0.6349843	total: 2.22s	remaining: 2m 36s
14:	learn: 0.6315508	total: 2.42s	remaining: 2m 38s
15:	learn: 0.6281954	total: 2.59s	remaining: 2m 39s
16:	learn: 0.6255600	total: 2.73s	remaining: 2m 37s
17:	learn: 0.6214640	total: 2.87s	remaining: 2m 36s
18:	learn: 0.6177454	total: 3.03s	remaining: 

157:	learn: 0.3817797	total: 24.7s	remaining: 2m 11s
158:	learn: 0.3808831	total: 24.8s	remaining: 2m 11s
159:	learn: 0.3798810	total: 25s	remaining: 2m 11s
160:	learn: 0.3793509	total: 25.2s	remaining: 2m 11s
161:	learn: 0.3783256	total: 25.3s	remaining: 2m 11s
162:	learn: 0.3775303	total: 25.5s	remaining: 2m 10s
163:	learn: 0.3768147	total: 25.6s	remaining: 2m 10s
164:	learn: 0.3761046	total: 25.8s	remaining: 2m 10s
165:	learn: 0.3751430	total: 25.9s	remaining: 2m 10s
166:	learn: 0.3741527	total: 26.1s	remaining: 2m 10s
167:	learn: 0.3730573	total: 26.2s	remaining: 2m 9s
168:	learn: 0.3722259	total: 26.4s	remaining: 2m 9s
169:	learn: 0.3709430	total: 26.5s	remaining: 2m 9s
170:	learn: 0.3702680	total: 26.7s	remaining: 2m 9s
171:	learn: 0.3691035	total: 26.8s	remaining: 2m 9s
172:	learn: 0.3683946	total: 27s	remaining: 2m 9s
173:	learn: 0.3676622	total: 27.2s	remaining: 2m 8s
174:	learn: 0.3668019	total: 27.3s	remaining: 2m 8s
175:	learn: 0.3661789	total: 27.4s	remaining: 2m 8s
176:	l

314:	learn: 0.2781340	total: 48.7s	remaining: 1m 45s
315:	learn: 0.2779179	total: 48.9s	remaining: 1m 45s
316:	learn: 0.2772433	total: 49.1s	remaining: 1m 45s
317:	learn: 0.2767178	total: 49.3s	remaining: 1m 45s
318:	learn: 0.2760170	total: 49.4s	remaining: 1m 45s
319:	learn: 0.2755604	total: 49.6s	remaining: 1m 45s
320:	learn: 0.2752095	total: 49.7s	remaining: 1m 45s
321:	learn: 0.2750147	total: 49.9s	remaining: 1m 45s
322:	learn: 0.2748417	total: 50.1s	remaining: 1m 44s
323:	learn: 0.2744210	total: 50.3s	remaining: 1m 44s
324:	learn: 0.2742264	total: 50.4s	remaining: 1m 44s
325:	learn: 0.2738204	total: 50.6s	remaining: 1m 44s
326:	learn: 0.2736523	total: 50.7s	remaining: 1m 44s
327:	learn: 0.2729670	total: 50.9s	remaining: 1m 44s
328:	learn: 0.2725383	total: 51s	remaining: 1m 44s
329:	learn: 0.2721704	total: 51.2s	remaining: 1m 43s
330:	learn: 0.2716721	total: 51.3s	remaining: 1m 43s
331:	learn: 0.2707995	total: 51.5s	remaining: 1m 43s
332:	learn: 0.2702408	total: 51.7s	remaining: 1m

469:	learn: 0.2120876	total: 1m 13s	remaining: 1m 23s
470:	learn: 0.2114623	total: 1m 13s	remaining: 1m 22s
471:	learn: 0.2110211	total: 1m 13s	remaining: 1m 22s
472:	learn: 0.2108346	total: 1m 14s	remaining: 1m 22s
473:	learn: 0.2101218	total: 1m 14s	remaining: 1m 22s
474:	learn: 0.2097329	total: 1m 14s	remaining: 1m 22s
475:	learn: 0.2095393	total: 1m 14s	remaining: 1m 22s
476:	learn: 0.2089368	total: 1m 14s	remaining: 1m 21s
477:	learn: 0.2086076	total: 1m 14s	remaining: 1m 21s
478:	learn: 0.2085079	total: 1m 15s	remaining: 1m 21s
479:	learn: 0.2080083	total: 1m 15s	remaining: 1m 21s
480:	learn: 0.2078195	total: 1m 15s	remaining: 1m 21s
481:	learn: 0.2077068	total: 1m 15s	remaining: 1m 21s
482:	learn: 0.2073473	total: 1m 15s	remaining: 1m 20s
483:	learn: 0.2071664	total: 1m 15s	remaining: 1m 20s
484:	learn: 0.2071188	total: 1m 15s	remaining: 1m 20s
485:	learn: 0.2069015	total: 1m 16s	remaining: 1m 20s
486:	learn: 0.2067370	total: 1m 16s	remaining: 1m 20s
487:	learn: 0.2064821	total:

624:	learn: 0.1588737	total: 1m 38s	remaining: 59.1s
625:	learn: 0.1587887	total: 1m 38s	remaining: 58.9s
626:	learn: 0.1585866	total: 1m 38s	remaining: 58.8s
627:	learn: 0.1581099	total: 1m 38s	remaining: 58.6s
628:	learn: 0.1577841	total: 1m 39s	remaining: 58.5s
629:	learn: 0.1574610	total: 1m 39s	remaining: 58.3s
630:	learn: 0.1572652	total: 1m 39s	remaining: 58.2s
631:	learn: 0.1569114	total: 1m 39s	remaining: 58s
632:	learn: 0.1568415	total: 1m 39s	remaining: 57.8s
633:	learn: 0.1565920	total: 1m 39s	remaining: 57.7s
634:	learn: 0.1560688	total: 1m 40s	remaining: 57.5s
635:	learn: 0.1557862	total: 1m 40s	remaining: 57.3s
636:	learn: 0.1553870	total: 1m 40s	remaining: 57.2s
637:	learn: 0.1550830	total: 1m 40s	remaining: 57s
638:	learn: 0.1547418	total: 1m 40s	remaining: 56.8s
639:	learn: 0.1543953	total: 1m 40s	remaining: 56.6s
640:	learn: 0.1542634	total: 1m 40s	remaining: 56.5s
641:	learn: 0.1539193	total: 1m 41s	remaining: 56.4s
642:	learn: 0.1536314	total: 1m 41s	remaining: 56.

...

499:	learn: 0.0833123	total: 1m 58s	remaining: 1m 58s
500:	learn: 0.0831023	total: 1m 59s	remaining: 1m 58s
501:	learn: 0.0827349	total: 1m 59s	remaining: 1m 58s
502:	learn: 0.0825765	total: 1m 59s	remaining: 1m 58s
503:	learn: 0.0824113	total: 1m 59s	remaining: 1m 57s
504:	learn: 0.0821661	total: 2m	remaining: 1m 57s
505:	learn: 0.0820943	total: 2m	remaining: 1m 57s
506:	learn: 0.0818049	total: 2m	remaining: 1m 57s
507:	learn: 0.0815566	total: 2m	remaining: 1m 56s
508:	learn: 0.0813497	total: 2m	remaining: 1m 56s
509:	learn: 0.0812257	total: 2m 1s	remaining: 1m 56s
510:	learn: 0.0810960	total: 2m 1s	remaining: 1m 56s
511:	learn: 0.0807553	total: 2m 1s	remaining: 1m 56s
512:	learn: 0.0807037	total: 2m 1s	remaining: 1m 55s
513:	learn: 0.0804868	total: 2m 2s	remaining: 1m 55s
514:	learn: 0.0803111	total: 2m 2s	remaining: 1m 55s
515:	learn: 0.0801318	total: 2m 2s	remaining: 1m 55s
516:	learn: 0.0800636	total: 2m 2s	remaining: 1m 54s
517:	learn: 0.0799457	total: 2m 3s	remaining: 1m 54s
518

652:	learn: 0.0577850	total: 2m 34s	remaining: 1m 21s
653:	learn: 0.0575213	total: 2m 34s	remaining: 1m 21s
654:	learn: 0.0572631	total: 2m 34s	remaining: 1m 21s
655:	learn: 0.0571872	total: 2m 34s	remaining: 1m 21s
656:	learn: 0.0569735	total: 2m 35s	remaining: 1m 20s
657:	learn: 0.0568401	total: 2m 35s	remaining: 1m 20s
658:	learn: 0.0566751	total: 2m 35s	remaining: 1m 20s
659:	learn: 0.0564698	total: 2m 35s	remaining: 1m 20s
660:	learn: 0.0563408	total: 2m 35s	remaining: 1m 19s
661:	learn: 0.0560996	total: 2m 36s	remaining: 1m 19s
662:	learn: 0.0560153	total: 2m 36s	remaining: 1m 19s
663:	learn: 0.0559445	total: 2m 36s	remaining: 1m 19s
664:	learn: 0.0557470	total: 2m 36s	remaining: 1m 18s
665:	learn: 0.0556757	total: 2m 36s	remaining: 1m 18s
666:	learn: 0.0555419	total: 2m 37s	remaining: 1m 18s
667:	learn: 0.0553351	total: 2m 37s	remaining: 1m 18s
668:	learn: 0.0551833	total: 2m 37s	remaining: 1m 17s
669:	learn: 0.0551206	total: 2m 37s	remaining: 1m 17s
670:	learn: 0.0550130	total:

808:	learn: 0.0382714	total: 3m 8s	remaining: 44.6s
809:	learn: 0.0382052	total: 3m 9s	remaining: 44.4s
810:	learn: 0.0381119	total: 3m 9s	remaining: 44.1s
811:	learn: 0.0379843	total: 3m 9s	remaining: 43.9s
812:	learn: 0.0379599	total: 3m 9s	remaining: 43.7s
813:	learn: 0.0378552	total: 3m 10s	remaining: 43.4s
814:	learn: 0.0377475	total: 3m 10s	remaining: 43.2s
815:	learn: 0.0376639	total: 3m 10s	remaining: 42.9s
816:	learn: 0.0376044	total: 3m 10s	remaining: 42.7s
817:	learn: 0.0375067	total: 3m 10s	remaining: 42.5s
818:	learn: 0.0373649	total: 3m 10s	remaining: 42.2s
819:	learn: 0.0373294	total: 3m 11s	remaining: 42s
820:	learn: 0.0372589	total: 3m 11s	remaining: 41.7s
821:	learn: 0.0371999	total: 3m 11s	remaining: 41.5s
822:	learn: 0.0371396	total: 3m 11s	remaining: 41.3s
823:	learn: 0.0370885	total: 3m 12s	remaining: 41.1s
824:	learn: 0.0370207	total: 3m 12s	remaining: 40.8s
825:	learn: 0.0368721	total: 3m 12s	remaining: 40.6s
826:	learn: 0.0368481	total: 3m 12s	remaining: 40.4s


964:	learn: 0.0274688	total: 3m 44s	remaining: 8.14s
965:	learn: 0.0274540	total: 3m 44s	remaining: 7.91s
966:	learn: 0.0273851	total: 3m 45s	remaining: 7.68s
967:	learn: 0.0273171	total: 3m 45s	remaining: 7.45s
968:	learn: 0.0272962	total: 3m 45s	remaining: 7.21s
969:	learn: 0.0272578	total: 3m 45s	remaining: 6.98s
970:	learn: 0.0271909	total: 3m 45s	remaining: 6.75s
971:	learn: 0.0271669	total: 3m 46s	remaining: 6.51s
972:	learn: 0.0271092	total: 3m 46s	remaining: 6.28s
973:	learn: 0.0270526	total: 3m 46s	remaining: 6.05s
974:	learn: 0.0269840	total: 3m 46s	remaining: 5.82s
975:	learn: 0.0269274	total: 3m 47s	remaining: 5.58s
976:	learn: 0.0268909	total: 3m 47s	remaining: 5.35s
977:	learn: 0.0268464	total: 3m 47s	remaining: 5.12s
978:	learn: 0.0268030	total: 3m 47s	remaining: 4.89s
979:	learn: 0.0267084	total: 3m 48s	remaining: 4.65s
980:	learn: 0.0266186	total: 3m 48s	remaining: 4.42s
981:	learn: 0.0265624	total: 3m 48s	remaining: 4.19s
982:	learn: 0.0264876	total: 3m 48s	remaining:

CatBoostError: c:/goagent/pipelines/buildmaster/catboost.gittt/catboost/libs/target/target_converter.cpp:53: Target value "False" cannot be parsed as float

In [13]:
print("\nf1\n",scores_f, "\nmean\n", sum(scores_f)/len(scores_f))
print("\nprecision\n",scores_p, "\nmean\n", sum(scores_p)/len(scores_p))
print("\nrecall\n",scores_r, "\nmean\n", sum(scores_r)/len(scores_r))
print("\nroc auc\n",scores_auc, "\nmean\n", sum(scores_auc)/len(scores_auc))

ZeroDivisionError: division by zero

In [14]:
? CatBoostClassifier

In [15]:
model = Pipeline(steps=[
    
   # ('anova', SelectKBest()),
    ('clf',  CatBoostClassifier(random_state=271828, verbose=0))])

grid_params = [{#'anova__k':[10, 20, 30, 40, 50,60, 70,80,90, 100],
    'clf__depth':[3,1,2,6,4,5,7,8,9,10],
          'clf__iterations':[250,100,500,1000],
          'clf__learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'clf__l2_leaf_reg':[3,1,5,10,100],
          #'clf__border_count':[32,5,10,20,50,100,200],
          #'ctr_leaf_count_limit':[50,5,10,20,100,200],
    'clf__scale_pos_weight':[0.1, 0.5, 1, 5, 10]}]


In [15]:
y_train.iloc[test]#, y_pred

,q6.1_INTRU,q6.2_DREAM,q6.3_FLASH,tred_cutoff
933,1.0,1.0,1.0,False
392,1.0,1.0,1.0,False
338,2.0,1.0,1.0,False
904,1.0,1.0,1.0,False
303,1.0,1.0,1.0,False
356,2.0,1.0,1.0,False
795,1.0,1.0,1.0,False
928,2.0,1.0,1.0,False
739,1.0,1.0,1.0,False
172,2.0,2.0,1.0,False


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, stratify=Y['tred_cutoff'], random_state=271828)
cv = StratifiedKFold(3, random_state=271828)

for activation in ['elu']: # ''
    for epochs in [250]: #200
        for loss_weights in [0.75, 0.5, 1,0.25, 3]: # 
            for iii in [1, 2, 3, 4, 5, 10]:
                dropout_rate = 0.9

                print("\nepochs=", epochs, "\nactivation=", activation, "\niii=", iii,
                      "\ndropout_rate =", dropout_rate, "\nloss_weight =", loss_weights)
                
                scores_f = []
                #scores_p = []
                #scores_r = []
                scores_auc = []
                
                for train, test in cv.split(X_train, y_train['tred_cutoff'].astype(int)):
                    X_train_cv = X_train.iloc[train]
                    y_train_cv = y_train.iloc[train]

                    visible = Input(shape=(100,))
                    x = Dense(50, activation=activation)(visible)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)


                    output_intrusion = Dense(1, activation='sigmoid')(x)
                    output_avoid = Dense(1, activation='sigmoid')(x)
                    output_hype = Dense(1, activation='sigmoid')(x)
                    output_50 = Dense(1, activation='sigmoid')(x)

                    es = EarlyStopping(monitor='val_loss', patience=50)

                    model = Model(inputs=visible, outputs=[output_intrusion,
                            output_avoid, output_hype, output_50])

                    model.compile(loss=["binary_crossentropy"] * 4,
                              optimizer='adam', loss_weights = [loss_weights] * 3 + [iii])
                    
                    t = [(y_train_cv['q6.1_INTRU']>2).values.astype(int), (y_train_cv['q6.2_DREAM']>2).values.astype(int),
                         (y_train_cv['q6.3_FLASH']>2).values.astype(int), (y_train_cv['tred_cutoff']).values.astype(int)]

                    
                    class_weight = [{0:1, 1:2}, {0:1, 1:2}, {0:1, 1:2}, {0:1, 1:4}]
                
                    
                    model.fit(X_train_cv, t, epochs=epochs, class_weight=class_weight, validation_split= 0.1, callbacks=[es], verbose=0)
                    y_pred = model.predict(X_train[features].iloc[test])[3] >= 0.5#).astype(int)
                    scores_f.append(f1_score(y_train.iloc[test]['tred_cutoff'], y_pred))
                 #   scores_p.append(precision_score(y_train.iloc[test], y_pred))
                  #  scores_r.append(recall_score(y_train.iloc[test], y_pred))
                    scores_auc.append(roc_auc_score(y_train.iloc[test]['tred_cutoff'], y_pred))
                ### results
                print("\n\nscores_f=",np.mean(scores_f))
                #print("scores_p=",scores_p)
                #print("scores_r=",scores_r)
                #print("scores_auc=",scores_auc)
                print("mean auc=",np.mean(scores_auc))


epochs= 250 
activation= elu 
iii= 1 
dropout_rate = 0.9 
loss_weight = 0.75


scores_f= 0.4277516953573292
mean auc= 0.658192569516557

epochs= 250 
activation= elu 
iii= 2 
dropout_rate = 0.9 
loss_weight = 0.75


scores_f= 0.4593861693861694
mean auc= 0.6796925118264682

epochs= 250 
activation= elu 
iii= 3 
dropout_rate = 0.9 
loss_weight = 0.75


scores_f= 0.4492689722158321
mean auc= 0.6768838698511596

epochs= 250 
activation= elu 
iii= 4 
dropout_rate = 0.9 
loss_weight = 0.75


scores_f= 0.44467368230021087
mean auc= 0.6743919464635976

epochs= 250 
activation= elu 
iii= 5 
dropout_rate = 0.9 
loss_weight = 0.75


In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
gs_2 = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
gs_2.fit(X[features], (Y['q6.2_DREAM']>2).values.astype(int))
print("roc_auc", gs_2.best_score_,"\nparams = ", gs_2.best_params_)

In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
gs_1 = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
gs_1.fit(X[features], (Y['q6.1_INTRU']>2).values.astype(int))
print("roc_auc", gs_1.best_score_,"\nparams = ", gs_1.best_params_)

In [ ]:
#xgb = XGBClassifier()
scv = StratifiedKFold(5)
tred_gs = GridSearchCV(model, grid_params,  scoring="roc_auc", cv=scv)
tred_gs.fit(X[features], (Y['tred_cutoff']).values.astype(int))
print("roc_auc", tred_gs.best_score_,"\nparams = ", tred_gs.best_params_)

In [ ]:

from sklearn.metrics import roc_curve, auc

#y_pred_only_avoidance = only_avoidance_gs.best_estimator_.predict_proba(X_test)
#y_pred_depression = depression_gs.best_estimator_.predict_proba(X_test)
#y_pred_tred = tred_gs.best_estimator_.predict_proba(X_test)
#y_pred_33 = regression_33_gs.best_estimator_.predict_proba(X_test)
y_pred_50 = regression_50_gs.best_estimator_.predict_proba(X_test)
y_pred_intrusion = intrusion_gs.best_estimator_.predict_proba(X_test)
y_pred_avoidance = avoidance_gs.best_estimator_.predict_proba(X_test)
y_pred_hypertension = hypertention_gs.best_estimator_.predict_proba(X_test)
preds = y_pred_50[:, 1]* y_pred_intrusion[:,1] * y_pred_avoidance[:,1] * y_pred_hypertension[:,1]

fpr, tpr, threshold = roc_curve(y_test, preds)
roc_auc = auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
plt.hist(preds[y_test==0], bins=50, color="b", label="non PTSD")
plt.hist(preds[y_test==1], color="r", label="PTSD")
plt.xlabel('pred probability')
plt.legend();
sorted(preds[y_test==1])